# Fundemental Tools

Tools used in fundemental analysis

In [35]:
#| default_exp fundemental

In [36]:
#| hide
from nbdev.showdoc import *

In [37]:
#| export
from typing import Union, Dict
import yfinance as yf
from langchain_core.tools import tool


@tool
def get_financial_metrics(ticker: str) -> Union[Dict, str]:
    """Fetches key financial ratios for a given ticker."""
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        return {
            'pe_ratio': info.get('forwardPE'),
            'price_to_book': info.get('priceToBook'),
            'debt_to_equity': info.get('debtToEquity'),
            'profit_margins': info.get('profitMargins')
        }
    except Exception as e:
        return f"Error fetching ratios: {str(e)}"

## Usage Example

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

FUNDAMENTAL_ANALYST_PROMPT = """
You are a fundamental analyst specializing in evaluating company (whose symbol is {company}) performance based on stock prices, technical indicators, and financial metrics. Your task is to provide a comprehensive summary of the fundamental analysis for a given stock.

You have access to the following tools:
1. **get_financial_metrics**: Retrieves key financial metrics, such as revenue, earnings per share (EPS), price-to-earnings ratio (P/E), and debt-to-equity ratio.

### Your Task:
1. **Input Stock Symbol**: Use the provided stock symbol to query the tools and gather the relevant information.
2. **Analyze Data**: Evaluate the results from the tools and identify potential resistance, key trends, strengths, or concerns.
3. **Provide Summary**: Write a concise, well-structured summary that highlights:
    - Recent stock price movements, trends and potential resistance.
    - Key insights from technical indicators (e.g., whether the stock is overbought or oversold).
    - Financial health and performance based on financial metrics.

### Constraints:
- Use only the data provided by the tools.
- Avoid speculative language; focus on observable data and trends.
- If any tool fails to provide data, clearly state that in your summary.

### Output Format:
Respond in the following format:
"stock": "<Stock Symbol>",
"price_analysis": "<Detailed analysis of stock price trends>",
"technical_analysis": "<Detailed time series Analysis from ALL technical indicators>",
"financial_analysis": "<Detailed analysis from financial metrics>",
"final Summary": "<Full Conclusion based on the above analyses>"
"Asked Question Answer": "<Answer based on the details and analysis above>"

Ensure that your response is objective, concise, and actionable.
"""

tools = [get_financial_metrics]

llm_with_tool = ChatOllama(
    model="llama3.2",
    temperature=0,
).bind_tools(tools=tools)

messages = [
    SystemMessage(content=FUNDAMENTAL_ANALYST_PROMPT.format(company="AAPL")),
    HumanMessage("Should I buy this stock?", pretty_repr=True),
    ]  

class State(TypedDict):
    messages: Annotated[list, add_messages]
    stock: str

def fundamental_analyst(state: State):
    messages = [
        SystemMessage(content=FUNDAMENTAL_ANALYST_PROMPT.format(company=state['stock'])),
    ]  + state['messages']
    return {
        'messages': llm_with_tool.invoke(messages)
    }

graph_builder = StateGraph(State)

graph_builder.add_node('fundamental_analyst', fundamental_analyst)
graph_builder.add_edge(START, 'fundamental_analyst')
graph_builder.add_node(ToolNode(tools))
graph_builder.add_conditional_edges('fundamental_analyst', tools_condition)
graph_builder.add_edge('tools', 'fundamental_analyst')

# HumanMessage(content=)
# graph_builder.add_edge('fundamental_analyst', END)
graph = graph_builder.compile()

# events = graph.stream({'messages':[('user', 'Should I buy this stock?')],
#  'stock': 'ALGM'}, stream_mode='values')
# for event in events:
#     if 'messages' in event:
#         event['messages'][-1].pretty_print()

================================ Human Message =================================

Should I buy this stock?
================================== Ai Message ==================================
Tool Calls:
  get_financial_metrics (0c43207a-394b-4e19-8ccd-6516b1e52f71)
 Call ID: 0c43207a-394b-4e19-8ccd-6516b1e52f71
  Args:
    ticker: ALGM
================================= Tool Message =================================
Name: get_financial_metrics

{"pe_ratio": 40.725807, "price_to_book": 4.980276, "debt_to_equity": 42.604, "profit_margins": -0.08453}
================================== Ai Message ==================================

Based on the financial metrics provided, here's a summary of the analysis:

**Recent Stock Price Movements and Trends:**
The current stock price is not available in the provided data. However, we can analyze the historical trends based on the given financial metrics.

**Key Insights from Technical Indicators:**
Unfortunately, no technical indicators were provided in

In [39]:
#| hide
import nbdev; nbdev.nbdev_export()